# Financial Transaction Manager

This notebook provides an interface for processing, categorizing, and managing financial transactions from CSV files. It adds "Business Type" and "Retailer" labels to transactions based on rules and user input.

In [154]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
from src.transaction_processor import TransactionProcessor

## 1. Load Transactions

Select a CSV file containing transaction data. The file should have the following columns:
- Status
- Date
- Description
- Debit
- Credit
- Member Name

In [155]:
# File selector widget
file_selector = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    layout=widgets.Layout(width='300px')
)

# Or use a dropdown to select from existing files
csv_files = [f for f in os.listdir('data') if f.endswith('.csv')]
file_dropdown = widgets.Dropdown(
    options=csv_files,
    description='Or select:',
    disabled=False,
    layout=widgets.Layout(width='300px')
)

# Initialize processor
processor = TransactionProcessor('src/category_db.json' if
os.path.exists('src/category_db.json') else None)

# Placeholder for transaction data
transactions_df = None

# Function to load the file
def load_file_from_upload(change):
    global transactions_df

    try:
        # Handle different ipywidgets FileUpload value formats
        # In newer versions it returns a tuple of FileInfo objects
        if isinstance(change['new'], tuple):
            if not change['new']:  # Empty tuple means no file selected
                return

            # Get the first file
            file_info = change['new'][0]

            # Write to temporary file
            with open('temp_upload.csv', 'wb') as f:
                f.write(file_info.content)

        # In older versions it returns a dict of file info
        elif isinstance(change['new'], dict) and len(change['new']) > 0:
            uploaded_file = list(change['new'].values())[0]
            content = uploaded_file['content']

            # Write to temporary file
            with open('temp_upload.csv', 'wb') as f:
                f.write(content)

        else:
            display(HTML("<b style='color:red'>No file uploaded or unsupported format</b>"))
            return

        # Load using processor
        transactions_df = processor.load_transactions('temp_upload.csv')
        display(HTML(f"<b>Successfully loaded {len(transactions_df)} transactions!</b>"))
        display(transactions_df)
    except Exception as e:
        display(HTML(f"<b style='color:red'>Error loading file: {str(e)}</b>"))
        import traceback
        print(traceback.format_exc())
    finally:
        # Clean up temp file
        if os.path.exists('temp_upload.csv'):
            os.remove('temp_upload.csv')

# Register callbacks
file_selector.observe(load_file_from_upload, names='value')

# Display widgets
display(widgets.HTML("<b>Upload a CSV file</b>"))
display(file_selector)

HTML(value='<b>Upload a CSV file</b>')

FileUpload(value=(), accept='.csv', description='Upload CSV', layout=Layout(width='300px'))

## 2. Automatic Categorization

Apply automatic categorization rules to the transactions.

In [156]:
# Button to trigger categorization
categorize_button = widgets.Button(
    description='Categorize Transactions',
    disabled=False,
    button_style='info',
    tooltip='Apply automatic categorization rules',
    icon='check'
)

# Function to categorize transactions
def categorize_transactions(b):
    global transactions_df
    
    if transactions_df is None or len(transactions_df) == 0:
        display(HTML("<b style='color:red'>No transactions loaded!</b>"))
        return
    
    # Apply categorization
    before_count = transactions_df[(transactions_df['Business Type'] != '') | (transactions_df['Retailer'] != '')].shape[0]
    transactions_df = processor.categorize_transactions(transactions_df)
    after_count = transactions_df[(transactions_df['Business Type'] != '') | (transactions_df['Retailer'] != '')].shape[0]
    
    # Display results
    newly_categorized = after_count - before_count
    display(HTML(f"<b>Categorized {newly_categorized} new transactions!</b>"))
    display(HTML(f"<b>Total categorized: {after_count} out of {len(transactions_df)}</b>"))
    
    # Show sample of categorized transactions
    # display(transactions_df[transactions_df['Business Type'] != ''].head())
    display(transactions_df)

# Register callback
categorize_button.on_click(categorize_transactions)

# Display button
display(categorize_button)

Button(button_style='info', description='Categorize Transactions', icon='check', style=ButtonStyle(), tooltip=…

## 3. Manual Categorization

Group similar uncategorized transactions and present them for manual labeling.

In [157]:
# Global variables for manual categorization
current_group = []
current_group_idx = 0
all_groups = []

# List of business types - initialized with defaults
business_types = ["Oakhurst", "Personal"]

# Button to find similar transactions
find_button = widgets.Button(
    description='Find Unlabeled Transactions',
    disabled=False,
    button_style='info',
    tooltip='Group similar unlabeled transactions',
    icon='search'
)

# Similarity threshold slider
similarity_slider = widgets.FloatSlider(
    value=0.6,
    min=0.1,
    max=0.9,
    step=0.1,
    description='Similarity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Function to find similar transactions
def find_similar_transactions(b):
    global transactions_df, all_groups, current_group_idx
    
    if transactions_df is None or len(transactions_df) == 0:
        display(HTML("<b style='color:red'>No transactions loaded!</b>"))
        return
    
    # Only consider transactions with unlabeled business type
    # Regardless of retailer status
    unlabeled_df = transactions_df[transactions_df["Business Type"] == ""]
    
    if unlabeled_df.empty:
        display(HTML("<b>No unlabeled business types found!</b>"))
        return
    
    # Find similar groups
    all_groups = processor.find_similar_descriptions(unlabeled_df, similarity_slider.value)
    
    # If no similar groups, create individual groups for each unlabeled transaction
    if not all_groups:
        display(HTML("<b>No similar transactions found. Showing all unlabeled transactions individually.</b>"))
        all_groups = [[idx] for idx in unlabeled_df.index]
    else:
        display(HTML(f"<b>Found {len(all_groups)} groups of similar transactions!</b>"))
    
    # Add any remaining unlabeled transactions that weren't grouped
    grouped_indices = set()
    for group in all_groups:
        grouped_indices.update(group)
    
    remaining_indices = set(unlabeled_df.index) - grouped_indices
    for idx in remaining_indices:
        all_groups.append([idx])
    
    current_group_idx = 0
    show_current_group()

# Register callback
find_button.on_click(find_similar_transactions)

# Create widgets for manual categorization
group_output = widgets.Output()

def show_current_group():
    global current_group, transactions_df, all_groups, current_group_idx, business_types
    
    with group_output:
        clear_output()
        
        if current_group_idx >= len(all_groups):
            display(HTML("<b>All unlabeled transactions processed!</b>"))
            return
        
        current_group = all_groups[current_group_idx]
        
        # Filter to show only transactions with unlabeled business type
        # regardless of retailer status
        unlabeled_indices = [idx for idx in current_group if transactions_df.loc[idx, "Business Type"] == ""]
        
        if not unlabeled_indices:
            # If all transactions in this group are labeled, move to next group
            current_group_idx += 1
            show_current_group()
            return
            
        # Display group info
        if len(unlabeled_indices) > 1:
            display(HTML(f"<h3>Group {current_group_idx + 1} of {len(all_groups)} ({len(unlabeled_indices)} similar transactions)</h3>"))
        else:
            display(HTML(f"<h3>Transaction {current_group_idx + 1} of {len(all_groups)}</h3>"))
        
        # Create checkboxes - all pre-selected by default
        checkboxes = []
        select_none_checkbox = widgets.Checkbox(
            value=False,
            description='Unselect All',
            disabled=False,
            indent=False
        )
        
        # Function to handle unselect all
        def on_select_none_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                for checkbox in checkboxes:
                    checkbox.value = not change['new']  # Invert the logic
        
        select_none_checkbox.observe(on_select_none_change)
        
        # Create a checkbox for each unlabeled transaction - pre-selected
        for i, idx in enumerate(unlabeled_indices):
            checkbox = widgets.Checkbox(
                value=True,  # Pre-selected
                description='',
                disabled=False,
                indent=False,
                layout=widgets.Layout(width='30px', min_width='30px', margin='0px')
            )
            checkboxes.append(checkbox)
        
        # Create business type selection with existing and default labels
        business_dropdown = widgets.Dropdown(
            options=[''] + sorted(business_types),
            description='Business Type:',
            disabled=False,
        )
        
        # New business type input
        new_business = widgets.Text(
            value='',
            placeholder='New business type',
            description='New Type:',
            disabled=False
        )
        
        # Label selected button
        label_button = widgets.Button(
            description='Label Selected',
            disabled=False,
            button_style='success',
            tooltip='Apply business type to selected transactions',
            icon='check'
        )
        
        # Next group button
        next_button = widgets.Button(
            description='Next Group',
            disabled=False,
            button_style='warning',
            tooltip='Go to next group',
            icon='forward'
        )
        
        # Function to label selected transactions
        def label_selected(b):
            global current_group_idx, business_types  # Ensure we use the global variables
            
            # Get selected business type
            business_type = business_dropdown.value or new_business.value
            
            # Get selected indices
            selected_indices = [unlabeled_indices[i] for i, checkbox in enumerate(checkboxes) if checkbox.value]
            
            if not selected_indices:
                display(HTML("<b style='color:orange'>No transactions selected!</b>"))
                return
                
            if not business_type:
                display(HTML("<b style='color:orange'>No business type specified!</b>"))
                return
            
            # If it's a new business type, add it to our list
            if business_type and business_type not in business_types:
                business_types.append(business_type)
            
            # Apply business type to selected transactions
            for idx in selected_indices:
                transactions_df.at[idx, "Business Type"] = business_type
            
            # Check if all transactions in this group now have a business type
            still_unlabeled = [idx for idx in current_group if transactions_df.loc[idx, "Business Type"] == ""]
            if not still_unlabeled:
                # All labeled, move to next group
                current_group_idx += 1
                show_current_group()
            else:
                # Refresh the current group view
                show_current_group()
        
        # Function to move to next group
        def next_group(b):
            global current_group_idx  # Ensure we use the global variable
            current_group_idx += 1
            show_current_group()
        
        # Register callbacks
        label_button.on_click(label_selected)
        next_button.on_click(next_group)
        
        # Display the unselect all checkbox
        display(select_none_checkbox)
        display(HTML("<hr style='margin: 5px 0;'>"))
        
        # Create header
        header = widgets.HTML("""
        <div style="display: flex; width: 100%; font-weight: bold; padding: 5px 0; margin-bottom: 8px; border-bottom: 2px solid #555;">
            <div style="width: 30px; min-width: 30px;">✓</div>
            <div style="width: 15%; padding-left: 5px;">Date</div>
            <div style="width: 65%; padding-left: 5px;">Description</div>
            <div style="width: 15%; padding-left: 5px;">Amount</div>
        </div>
        """)
        display(header)
        
        # Create a list of rows, each with a checkbox and transaction info
        for i, idx in enumerate(unlabeled_indices):
            row = transactions_df.loc[idx]
            date = row['Date']
            amount = f"${row['Debit'] if pd.notna(row['Debit']) else row['Credit']}"
            
            # Display retailer value if present
            retailer_info = ""
            if row["Retailer"]:
                retailer_info = f" <span style='color: green;'>(Retailer: {row['Retailer']})</span>"
            
            # Create a wrapper div for the checkbox
            checkbox_wrapper = widgets.Box([checkboxes[i]], 
                                        layout=widgets.Layout(
                                            width='30px',
                                            min_width='30px',
                                            padding='0px',
                                            margin='0px',
                                            overflow='visible'
                                        ))
            
            # Create the date, description, and amount cells
            date_cell = widgets.HTML(f"<div style='padding: 3px'>{date}</div>",
                                  layout=widgets.Layout(width='15%', min_width='80px'))
            
            desc_cell = widgets.HTML(f"<div style='padding: 3px'>{row['Description']}{retailer_info}</div>",
                                  layout=widgets.Layout(width='65%'))
            
            amount_cell = widgets.HTML(f"<div style='padding: 3px'>{amount}</div>",
                                    layout=widgets.Layout(width='15%', min_width='80px'))
            
            # Create a horizontal box for the row
            row_box = widgets.HBox([
                checkbox_wrapper, 
                date_cell,
                desc_cell,
                amount_cell
            ], layout=widgets.Layout(
                width='100%',
                border='1px solid #ddd',
                margin='2px 0',
                padding='5px 0',
                align_items='center'
            ))
            
            display(row_box)
        
        display(HTML("<hr>"))
        
        # Display labeling controls
        display(widgets.HTML("<b>Apply a business type to selected transactions:</b>"))
        display(widgets.HBox([business_dropdown, new_business]))
        display(widgets.HBox([label_button, next_button]))

# Display widgets
display(widgets.HBox([find_button, similarity_slider]))
display(group_output)

Output()

## 4. Save Results

Save the categorized transactions to a CSV file.

In [158]:
# Output filename
output_filename = widgets.Text(
    value='categorized_transactions.csv',
    placeholder='output filename',
    description='Filename:',
    disabled=False
)

# Save button
save_button = widgets.Button(
    description='Save Transactions',
    disabled=False,
    button_style='success',
    tooltip='Save categorized transactions to a CSV file',
    icon='save'
)

# Function to save transactions
def save_transactions(b):
    global transactions_df
    
    if transactions_df is None or len(transactions_df) == 0:
        display(HTML("<b style='color:red'>No transactions to save!</b>"))
        return
    
    filename = output_filename.value
    if not filename.endswith('.csv'):
        filename += '.csv'
    
    # Create output directory if it doesn't exist
    output_dir = 'output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_path = os.path.join(output_dir, filename)
    
    # Save using processor
    processor.save_transactions(transactions_df, output_path)
    display(HTML(f"<b style='color:green'>Saved {len(transactions_df)} transactions to {output_path}!</b>"))

# Register callback
save_button.on_click(save_transactions)

# Display widgets
display(widgets.HBox([output_filename, save_button]))

## 5. Transaction Statistics

View statistics about the categorized transactions.

In [159]:
# Button to show statistics
stats_button = widgets.Button(
    description='Show Statistics',
    disabled=False,
    button_style='info',
    tooltip='Show statistics about categorized transactions',
    icon='bar-chart'
)

# Output for statistics
stats_output = widgets.Output()

# Function to show statistics
def show_statistics(b):
    global transactions_df
    
    if transactions_df is None or len(transactions_df) == 0:
        with stats_output:
            clear_output()
            display(HTML("<b style='color:red'>No transactions loaded!</b>"))
        return
    
    with stats_output:
        clear_output()
        
        # Calculate statistics
        total = len(transactions_df)
        categorized_business = transactions_df[transactions_df['Business Type'] != ''].shape[0]
        uncategorized_business = total - categorized_business
        
        categorized_retailer = transactions_df[transactions_df['Retailer'] != ''].shape[0]
        uncategorized_retailer = total - categorized_retailer
        
        # Display basic statistics
        display(HTML(f"<h3>Transaction Statistics</h3>"))
        display(HTML(f"<b>Total transactions: {total}</b>"))
        display(HTML(f"<b>Business Type: {categorized_business} categorized ({categorized_business/total*100:.1f}%), {uncategorized_business} unlabeled ({uncategorized_business/total*100:.1f}%)</b>"))
        display(HTML(f"<b>Retailer: {categorized_retailer} categorized ({categorized_retailer/total*100:.1f}%), {uncategorized_retailer} unlabeled ({uncategorized_retailer/total*100:.1f}%)</b>"))
        
        # Create subplots for charts
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Business Type distribution - Include unlabeled
        business_counts = transactions_df['Business Type'].value_counts(dropna=False)
        # Rename empty string to 'Unlabeled'
        if '' in business_counts:
            business_counts = business_counts.rename({'': 'Unlabeled'})
        
        if not business_counts.empty:
            # Use a custom color map with red for unlabeled
            colors = plt.cm.tab10(range(len(business_counts)))
            if 'Unlabeled' in business_counts.index:
                # Find the index of 'Unlabeled' and set its color to light gray
                unlabeled_idx = list(business_counts.index).index('Unlabeled')
                colors[unlabeled_idx] = [0.8, 0.8, 0.8, 1.0]  # Light gray
            
            # Create the pie chart with custom colors
            wedges, texts, autotexts = ax1.pie(
                business_counts, 
                labels=business_counts.index, 
                autopct='%1.1f%%', 
                colors=colors,
                startangle=90
            )
            
            # Make the 'Unlabeled' label and percentage italic
            if 'Unlabeled' in business_counts.index:
                for i, label in enumerate(business_counts.index):
                    if label == 'Unlabeled':
                        texts[i].set_style('italic')
                        autotexts[i].set_style('italic')
                        
            ax1.set_title('Business Type Distribution')
            ax1.set_ylabel('')
        else:
            ax1.text(0.5, 0.5, 'No Business Types assigned', ha='center', va='center')
            ax1.set_title('Business Type Distribution')
        
        # Retailer distribution - Include unlabeled
        retailer_counts = transactions_df['Retailer'].value_counts(dropna=False)
        # Rename empty string to 'Unlabeled'
        if '' in retailer_counts:
            retailer_counts = retailer_counts.rename({'': 'Unlabeled'})
        
        if not retailer_counts.empty:
            # Use a custom color map with red for unlabeled
            colors = plt.cm.tab10(range(len(retailer_counts)))
            if 'Unlabeled' in retailer_counts.index:
                # Find the index of 'Unlabeled' and set its color to light gray
                unlabeled_idx = list(retailer_counts.index).index('Unlabeled')
                colors[unlabeled_idx] = [0.8, 0.8, 0.8, 1.0]  # Light gray
            
            # Create the pie chart with custom colors
            wedges, texts, autotexts = ax2.pie(
                retailer_counts, 
                labels=retailer_counts.index, 
                autopct='%1.1f%%',
                colors=colors,
                startangle=90
            )
            
            # Make the 'Unlabeled' label and percentage italic
            if 'Unlabeled' in retailer_counts.index:
                for i, label in enumerate(retailer_counts.index):
                    if label == 'Unlabeled':
                        texts[i].set_style('italic')
                        autotexts[i].set_style('italic')
                        
            ax2.set_title('Retailer Distribution')
            ax2.set_ylabel('')
        else:
            ax2.text(0.5, 0.5, 'No Retailers assigned', ha='center', va='center')
            ax2.set_title('Retailer Distribution')
        
        # Adjust legend and layout
        plt.tight_layout()
        plt.show()

# Register callback
stats_button.on_click(show_statistics)

# Display widgets
display(stats_button)
display(stats_output)

Button(button_style='info', description='Show Statistics', icon='bar-chart', style=ButtonStyle(), tooltip='Sho…

Output()